Во рамки на оваа вежба ќе работиме со два модели, односно на два начини ќе направиме конверзија од говор во текст.

Најпрво, да го погледнеме првиот начин.

## Huggingface Transformers

Huggingface е open-source provider за NLP технологии (технологии за процесирање на природни јазици). 

Huggingface transformers е пакет на популарни Python библиотеки кои овозможуваат користење на различни претренирани модели кои ни се од неверојатна помош при различни NLP задачи.

Во случајот ќе искористиме претрениран модел за кој можете да најдете повеќе детали [на овој линк](https://huggingface.co/transformers/model_doc/wav2vec2.html).

Моделот кој ќе го користиме работи со Англискиот јазик, и е трениран врз податоци од Facebook, па може да не даде добри резултати доколку истиот ни е потребен за некој специфичен домен.

In [ ]:
!pip install transformers

In [ ]:
import torch
import librosa
import numpy as np
import soundfile as sf
from scipy.io import wavfile
from IPython.display import Audio
from transformers import Wav2Vec2ForCTC, Wav2Vec2Tokenizer

In [ ]:
tokenizer = Wav2Vec2Tokenizer.from_pretrained("facebook/wav2vec2-base-960h")
model = Wav2Vec2ForCTC.from_pretrained("facebook/wav2vec2-base-960h")

Downloading:   0%|          | 0.00/291 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/163 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/85.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.56k [00:00<?, ?B/s]

The tokenizer class you load from this checkpoint is not the same type as the class this function is called from. It may result in unexpected tokenization. 
The tokenizer class you load from this checkpoint is 'Wav2Vec2CTCTokenizer'. 
The class this function is called from is 'Wav2Vec2Tokenizer'.
/usr/local/lib/python3.7/dist-packages/transformers/models/wav2vec2/tokenization_wav2vec2.py:423: FutureWarning: The class `Wav2Vec2Tokenizer` is deprecated and will be removed in version 5 of Transformers. Please use `Wav2Vec2Processor` or `Wav2Vec2CTCTokenizer` instead.
  FutureWarning,


Downloading:   0%|          | 0.00/360M [00:00<?, ?B/s]

Some weights of Wav2Vec2ForCTC were not initialized from the model checkpoint at facebook/wav2vec2-base-960h and are newly initialized: ['wav2vec2.masked_spec_embed']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


За овој дел сакаме да го искористиме аудио фајлот кој претходно го креиравме во првата вежба, но истиот ни е потребен во wav формат, па ќе го конвертираме.

За да го направиме ова, најпрво ќе го симнеме фајлот кој го креиравме во првата вежба. Потоа во делот Files -> Upload ќе го ставиме овој фајл.

За крај го конвертираме во wav формат.

In [ ]:
!pip install pydub

За да го конвертираме фајлот користиме pydub библиотека.

In [ ]:
from pydub import AudioSegment
                                                                      
src = "first.mp3"
dst = "first.wav"                                                        
audSeg = AudioSegment.from_mp3(src)
audSeg.export(dst, format="wav")

<_io.BufferedRandom name='first.wav'>

Следно, го читаме фајлот и го користиме tokenizer-от на готовиот модел. На крај ја декодираме транскрипцијата.

In [ ]:
file_name = 'first.wav'
data = wavfile.read(file_name)
framerate = data[0]
sounddata = data[1]

time = np.arange(0,len(sounddata))/framerate
input_audio, _ = librosa.load(file_name, sr=16000)
input_values = tokenizer(input_audio, return_tensors="pt").input_values

logits = model(input_values).logits
predicted_ids = torch.argmax(logits, dim=-1)
transcription = tokenizer.batch_decode(predicted_ids)[0]

print(transcription)

THIS IS EXCITING


## SpeechRecognition

Втор начин на конверзија од говор во текст ќе биде со помош на SpeechRecognition.

Прв чекор ќе биде да го симнеме модулот кој го користиме.

In [ ]:
!pip install SpeechRecognition

SpeechRecognition е библиотека која ни овозможува распознавање на говор и негова конверзија во текст. Истата поддржува неколку јазици, меѓу кои и Англискиот јазик.

[На овој линк](https://pypi.org/project/SpeechRecognition/) може да погледнете детали за самата библиотека.

Следните чекори ни се:

*   да ја импортираме библиотеката

*   да ја иницијализираме класата за самиот recogniser за да распознава говор

*   да искористиме google speech recognition







In [ ]:
import speech_recognition as sr

r = sr.Recognizer()


with sr.AudioFile('first.wav') as source:
    audio_text = r.listen(source)
    
    try:
        text = r.recognize_google(audio_text)
        print(text)
     
    except:
         print('There was an error. Try again ...')

this is exciting


Доколку не специфицираме друг јазик, тогаш се користи Англискиот јазик. Во случај да сакаме да специфицираме јазик, тоа можеме да го направиме со следниот код.

Во случајот се специфицира Француски јазик.

In [ ]:
#text = r.recognize_google(audio_text, language = "fr-FR")

Со ова разработивме успешно два готови модели за говор во текст!